<a href="https://colab.research.google.com/github/Nburkhal/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [7]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np

data_link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

vote_df = pd.read_csv(data_link, header=None)
vote_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [10]:
# Replace ? with NaN

# Check to see if there are any leading/trailing whitespaces
vote_df[1][1]

'n'

In [0]:
# Replace ? with np.nan
vote_df = vote_df.replace('?', np.nan)

In [14]:
# Add column names to dataframe
vote_df.columns = ['party', 'handicapped_infants', 'water_project_cost_sharing', 
                  'adoption_of_budget_resolution', 'physician_fee_freeze', 
                  'el_salvador_aid', 'religious_groups_in_schools', 
                  'anti_satellite_test_ban', 'aid_to_nicaraguan_contras', 
                  'mx_missile', 'immigration', 'synfuels_corp_cutback', 
                  'education_spending', 'superfund_right_to_sue', 'crime', 
                  'duty_free_exports', 'export_admin_act_south_africa']

vote_df.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_admin_act_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [26]:
# Change y & n to 1 & 0
vote_df = vote_df.replace(['y', 'n'], [1, 0])

vote_df.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_admin_act_south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [33]:
# Check dataframe dtypes
vote_df.dtypes

party                             object
handicapped_infants              float64
water_project_cost_sharing       float64
adoption_of_budget_resolution    float64
physician_fee_freeze             float64
el_salvador_aid                  float64
religious_groups_in_schools      float64
anti_satellite_test_ban          float64
aid_to_nicaraguan_contras        float64
mx_missile                       float64
immigration                      float64
synfuels_corp_cutback            float64
education_spending               float64
superfund_right_to_sue           float64
crime                            float64
duty_free_exports                float64
export_admin_act_south_africa    float64
dtype: object

In [0]:
# Create political party samples
gop = vote_df[vote_df['party'] == 'republican']
dems = vote_df[vote_df['party'] == 'democrat']

In [34]:
print(gop.dtypes)
gop.head()

party                             object
handicapped_infants              float64
water_project_cost_sharing       float64
adoption_of_budget_resolution    float64
physician_fee_freeze             float64
el_salvador_aid                  float64
religious_groups_in_schools      float64
anti_satellite_test_ban          float64
aid_to_nicaraguan_contras        float64
mx_missile                       float64
immigration                      float64
synfuels_corp_cutback            float64
education_spending               float64
superfund_right_to_sue           float64
crime                            float64
duty_free_exports                float64
export_admin_act_south_africa    float64
dtype: object


,party,handicapped_infants,water_project_cost_sharing,adoption_of_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_admin_act_south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [35]:
print(dems.dtypes)
dems.head()

party                             object
handicapped_infants              float64
water_project_cost_sharing       float64
adoption_of_budget_resolution    float64
physician_fee_freeze             float64
el_salvador_aid                  float64
religious_groups_in_schools      float64
anti_satellite_test_ban          float64
aid_to_nicaraguan_contras        float64
mx_missile                       float64
immigration                      float64
synfuels_corp_cutback            float64
education_spending               float64
superfund_right_to_sue           float64
crime                            float64
duty_free_exports                float64
export_admin_act_south_africa    float64
dtype: object


,party,handicapped_infants,water_project_cost_sharing,adoption_of_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_admin_act_south_africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [30]:
# Check descriptive statistics - see which parties voted for more on what issues

gop.describe()

,handicapped_infants,water_project_cost_sharing,adoption_of_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_admin_act_south_africa
count,165.000000,148.000000,164.000000,165.000000,165.000000,166.000000,162.000000,157.000000,165.000000,165.000000,159.000000,155.000000,158.000000,161.000000,156.000000,146.000000
mean,0.187879,0.506757,0.134146,0.987879,0.951515,0.897590,0.240741,0.152866,0.115152,0.557576,0.132075,0.870968,0.860759,0.981366,0.089744,0.657534
std,0.391804,0.501652,0.341853,0.109760,0.215442,0.304104,0.428859,0.361010,0.320176,0.498186,0.339643,0.336322,0.347298,0.135649,0.286735,0.476168
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
dems.describe()

,handicapped_infants,water_project_cost_sharing,adoption_of_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_admin_act_south_africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# Define function to calculate 2 sample t-tests
def two_tail_ttest(x, y, z='omit'):
  
  from scipy.stats import ttest_ind
  
  return ttest_ind(x, y, nan_policy=z)

In [41]:
# 2 sample t-test for the adoption of the budget
# mean democrat = 0.88
# mean republican = 0.13
# p-value = 0.01

from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

ttest_ind(dems['adoption_of_budget_resolution'], gop['adoption_of_budget_resolution'], nan_policy='omit')
two_tail_ttest(dems['adoption_of_budget_resolution'], gop['adoption_of_budget_resolution'])

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

Due to calculating a t-statistic of 23.2127 which corresponds to a p-value of 2.07e-77, we reject the null hypothesis, using a 99% confidence interval, that the mean of votes between the deomcrats and republicans regarding the adoption of a budget resolution is equal, and suggest the alternative hypothesis that they are different.

In [42]:
# 2 sample t-test for religious groups in schools
# mean democrat = 0.477
# mean republican = 0.898
# p-value = 0.01

ttest_ind(gop['religious_groups_in_schools'], dems['religious_groups_in_schools'], nan_policy='omit')

two_tail_ttest(gop['religious_groups_in_schools'], dems['religious_groups_in_schools'])

Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)

Due to calculating a t-statistic of 9.7376 which corresponds to a p-value of 2.39e-20, we reject the null hypothesis, using a 99% confidence interval, that the mean of votes between the deomcrats and republicans regarding religious groups in schools is equal, and suggest the alternative hypothesis that they are different.

In [38]:
# 2 sample t-test for water project cost sharing
# mean democrat = 0.502
# mean republican = 0.507
# p-value = 0.01

ttest_ind(dems['water_project_cost_sharing'], gop['water_project_cost_sharing'], nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

Due to calculating a t-statistic of -0.0889 which corresponds to a p-value of 0.9291, we fail to reject the null hypothesis, using a 99% confidence interval, that the mean of votes between the deomcrats and republicans regarding water project cost sharing.

In [43]:
# Testing to see if our function truly works
two_tail_ttest(gop['crime'], dems['crime'])

Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47)

Due to calculating a t-statistic of 16.34 which corresponds to a p-value of 9.95e-47, we reject the null hypothesis, using a 99% confidence interval, that the mean of votes between the republicans and democrats regarding crime is equal, and suggest the alternative hypothesis that they are different.

In [0]:
# Define 1-sample t-test
def one_sample_ttest(data, popmean, z='omit'):
  
  from scipy.stats import ttest_1samp
  
  return ttest_1samp(data, popmean, nan_policy=z)

In [45]:
# Perform one_sample t-test function test
# Do democrats favor synfuals corporate cutbacks, or is the vote split down the middle?

one_sample_ttest(dems['synfuels_corp_cutback'], 0.5)

Ttest_1sampResult(statistic=0.18751153526716705, pvalue=0.8514092819865365)

Due to calculating a t-statistic of 0.1875 which corresponds to a p-value of 0.8514, we fail to reject the null hypothesis, using a 99% confidence interval, that the mean of votes among deomcrats regarding synfuels corporate cutbacks does not sway in a particular direction.